In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
from typing import Optional, cast, Dict, Any

import torch

import transformers
from transformers import AutoModel, AutoConfig, AutoTokenizer

from omegaconf import DictConfig
from omegaconf import OmegaConf as om

from src.flex_bert import *
from src.evals.data import *

/home/azureuser/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'src'

In [3]:
def consume_prefix_in_state_dict_if_present(
    state_dict, prefix
):
    r"""Strip the prefix in state_dict in place, if any.

    ..note::
        Given a `state_dict` from a DP/DDP model, a local model can load it by applying
        `consume_prefix_in_state_dict_if_present(state_dict, "module.")` before calling
        :meth:`torch.nn.Module.load_state_dict`.

    Args:
        state_dict (OrderedDict): a state-dict to be loaded to the model.
        prefix (str): prefix.
    """
    keys = sorted(state_dict.keys())
    for key in keys:
        if key.startswith(prefix):
            newkey = key[len(prefix) :]
            state_dict[newkey] = state_dict.pop(key)

    # also strip the prefix in metadata if any.
    if "_metadata" in state_dict:
        metadata = state_dict["_metadata"]
        for key in list(metadata.keys()):
            # for the metadata dict, the key can be:
            # '': for the DDP module, which we want to remove.
            # 'module': for the actual model.
            # 'module.xx.xx': for the rest.

            if len(key) == 0:
                continue
            newkey = key[len(prefix) :]
            metadata[newkey] = metadata.pop(key)

In [4]:
state_dict = torch.load("/home/public/span/MATH_DPO/modern_bert_test/bert24/checkpoints/ep1-ba542-rank0.pt")
state_dict = state_dict['state']['model']
consume_prefix_in_state_dict_if_present(state_dict, "model.")
torch.save(state_dict, "/home/public/span/MATH_DPO/modern_bert_test/bert24/checkpoints/correct_names.pt")

/home/public/span/temp/ipykernel_35250/1181012206.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/home/public/span/MATH_DPO/modern_bert_test/be

In [ ]:
with open("/home/public/span/MATH_DPO/modern_bert_test/bert24/yamls/test/sequence_classification_og.yaml") as f:
    yaml_config = om.load(f)

cfg = cast(DictConfig, yaml_config)

In [6]:
model = create_flex_bert_classification(
    num_labels=cfg.model.num_labels,
    pretrained_checkpoint=cfg.model.pretrained_checkpoint,
    model_config=cfg.model.model_config,
    tokenizer_name=cfg.tokenizer_name
)

/home/public/span/MATH_DPO/modern_bert_test/bert24/src/bert_layers/model.py:1300: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(pretrained_checkpoint

In [2]:
import datasets
ds = datasets.load_dataset("sarahpann/reward_bench_processed")

Generating train split: 100%|██████████| 5123/5123 [00:00<00:00, 212578.35 examples/s]


In [5]:
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-large")
model = transformers.AutoModelForMaskedLM.from_pretrained("sarahpann/all_at_once_model")

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [6]:
dataloader = torch.utils.data.DataLoader(ds['train'])

In [7]:
all_dsets = {'llmbar-adver-manual', 'hep-python', 'refusals-dangerous', 
                 'mt-bench-med', 'refusals-offensive', 'alpacaeval-length', 
                 'llmbar-natural', 'alpacaeval-hard', 'mt-bench-easy', 
                 'mt-bench-hard', 'llmbar-adver-GPTOut', 'hep-rust', 
                 'hep-java', 'donotanswer', 'hep-js', 'llmbar-adver-GPTInst',
                   'math-prm', 'hep-go', 'alpacaeval-easy', 'xstest-should-respond', 
                   'llmbar-adver-neighbor', 'hep-cpp', 'xstest-should-refuse'}

categories = {
    "chat": {'alpacaeval-easy', 'alpacaeval-length', 'alpacaeval-hard', 
                'mt-bench-easy', 'mt-bench-med'},
    "chat_hard": {'mt-bench-hard', 'llmbar-natural', 'llmbar-adver-neighbor',
                    'llmbar-adver-GPTInst', 'llmbar-adver-GPTOut', 'llmbar-adver-manual'},
    "safety": {'refusals-dangerous', 'refusals-offensive', 'xstest-should-respond', 
                'xstest-should-refuse', 'xstest-should-respond', 'donotanswer'},
    "reasoning": {'math-prm', 'hep-cpp', 'hep-go', 'hep-java', 'hep-js', 'hep-python', 'hep-rust'},
}

inverse_map = {v: k for k in categories for v in categories[k]}
category_tallies = {k: 0 for k in categories.keys()}
category_amounts = {k: 0 for k in categories.keys()}

In [8]:
for ex in dataloader:
    print(ex)
    break

{'chosen': ['What are the names of some famous actors that started their careers on Broadway?\nSeveral famous actors started their careers on Broadway before making it big in film and television. Here are a few notable examples:\n\n1. Sarah Jessica Parker - Before she was Carrie Bradshaw on "Sex and the City," Sarah Jessica Parker was a Broadway star, having appeared in productions like "Annie" as a child.\n\n2. Meryl Streep - Meryl Streep\'s early career included Broadway productions such as "Trelawny of the \'Wells\'" and "A Memory of Two Mondays / 27 Wagons Full of Cotton."\n\n3. Hugh Jackman - Hugh Jackman won a Tony Award for his role in "The Boy from Oz" and has been known for his stage work as well as his film career.\n\n4. Sutton Foster - Known for her television role in "Younger," Sutton Foster is also a Broadway legend with leading roles in shows like "Thoroughly Modern Millie" and "Anything Goes."\n\n5. Kristen Bell - Before she was the voice of Anna in "Frozen" or the star 

In [16]:
model = model.model

In [9]:
for ex in dataloader:
    print(ex.keys())
    break

dict_keys(['chosen', 'rejected', 'og_dataset'])


In [12]:
    def tokenize_fn_factory(tokenizer, max_seq_length):
        tokenized_postfix = tokenizer(
            text=postfix,
            padding="max_length",
            max_length=max_seq_length,
            truncation=True,
        )['input_ids'][1:]
        len_tokenized_postfix = len(tokenized_postfix)

        def tokenizer_fn(inp):
            # flip a coin
            coin = random.randint(0, 1)
            # if coin == 0, then chosen goes first
            choice_postfix = postfix + " " + str(coin)

            if coin == 0:
                pairs = [[prefix + " Choice 0: " + chosen, " Choice 1: " + rejected + choice_postfix] for chosen, rejected in zip(inp["chosen"], inp["rejected"])]
            if coin == 1:
                pairs = [[prefix + " Choice 0: " + rejected, " Choice 1: " + chosen + choice_postfix] for chosen, rejected in zip(inp["chosen"], inp["rejected"])]

            tokenized_pairs = tokenizer(pairs,
                                        max_length=max_seq_length, 
                                        truncation=True)

            # Always has an answer even if truncated
            if len(tokenized_pairs["input_ids"]) == max_seq_length:
                tokenized_pairs["input_ids"] = tokenized_pairs["input_ids"][:-(len_tokenized_postfix)] + tokenized_postfix

            ret_dict = {
                "input_ids": tokenized_pairs["input_ids"],
                "token_type_ids": tokenized_pairs["token_type_ids"],
                "attention_mask": tokenized_pairs["attention_mask"],
            }

            return ret_dict
        
        return tokenizer_fn

In [17]:
columns_to_remove = [i for i in ['chosen', 'rejected', 'og_dataset'] if i is not None]

ds = ds.map(
    tokenize_fn_factory(tokenizer, 3600),
    batched=True,
    # batched=False,
    num_proc=None if num_workers == 0 else num_workers,
    batch_size=200,
    remove_columns=columns_to_remove,
    load_from_cache_file=True,
    )

NameError: name 'postfix' is not defined

In [ ]:
# model = model.model
model.to("cuda")
model.eval()

with torch.no_grad():
    for example in dataloader:
        tokenized_chosen = tokenizer(example['chosen'], return_tensors="pt")
        tokenized_rejected = tokenizer(example['rejected'], return_tensors="pt")

        chosen_out = model(tokenized_chosen['input_ids'].to("cuda"))
        rejected_out = model(tokenized_rejected['input_ids'].to("cuda"))

        chosen_pred = torch.argmax(chosen_out.logits, dim=1)
        rejected_pred = torch.argmax(rejected_out.logits, dim=1)

        print(chosen_pred)
        print(rejected_pred)

        if chosen_pred == 1:
            category_tallies[inverse_map[example['og_dataset'][0]]] += 1

        if rejected_pred == 0:
            category_tallies[inverse_map[example['og_dataset'][0]]] += 1

        category_amounts[inverse_map[example['og_dataset'][0]]] += 2


print(f"{category_tallies=}")

# get accuracies
accuracy_dict = {k: category_tallies[k] / category_amounts[k] for k in category_tallies.keys()}
print(f"{accuracy_dict=}")

# get total accuracy
total_correct = sum(category_tallies.values())
total_amount = sum(category_amounts.values())
total_accuracy = total_correct / total_amount
print(f"{total_accuracy=}")

/home/azureuser/miniconda3/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:124: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


RuntimeError: Boolean value of Tensor with more than one value is ambiguous